In [2]:
import pandas as pd
import csv
import math
import clevercsv

csv.field_size_limit(10**6*10)

131072

In [1]:
import psycopg2
import csv

# Replace with your PostgreSQL connection URL
db_url = ""

# SQL query to select the columns you want
query = "SELECT index, story_id FROM doc_embeddings"

# File path for the single CSV file
output_file_path = "doc_embeddings_combined_v2.csv"

# Batch size
batch_size = 10000

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(db_url)

# Create a cursor
cursor = conn.cursor()

# Execute the query
cursor.execute(query)

# Open the CSV file for writing
with open(output_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write header
    csv_writer.writerow(["index", "story_id"])

    batch_number = 0
    while True:
        results = cursor.fetchmany(batch_size)
        if not results:
            break

        # Write data for the current batch
        csv_writer.writerows(results)

        batch_number += 1

# Close the cursor and the connection
cursor.close()
conn.close()

print(f"Data has been exported to {output_file_path}")

Data has been exported to doc_embeddings_combined_v2.csv


In [3]:
already_loaded_rows = clevercsv.read_dataframe('./doc_embeddings_combined_v2.csv', num_chars=1000000)

In [4]:
original_rows = pd.read_pickle('./cleaned_normalized_df_no_grouping.pkl')

In [5]:
original_rows['FictionId'] = original_rows['FictionId'].astype(int)
original_rows['Order'] = original_rows['Order'].astype(int)

original_rows.drop(columns=['ChapterId'], inplace=True)

original_rows

,FictionId,content,Order
0,24,"<p>Lorem ipsum dolor sit amet, consectetur adi...",1
1,24,<div>\n<p><strong>Lorem Ipsum</strong> is simp...,3
2,24,<p>Go away. Let me test stuff in peace!</p>\n<...,2
3,24,"<table style=""border-collapse: collapse; width...",4
4,24,<p>Nullam quis ante. Donec vitae orci sed dolo...,5
...,...,...,...
1243341,9997,"<p>Have you ever wonder, what would an apocaly...",1
1243342,9997,"<p>In a space full of nothingness, where there...",2
1243343,9997,"<p>Calm, almost tranquil atmosphere. That is V...",3
1243344,9998,<p>“It has been a long time since one of your ...,1


In [6]:
already_loaded_rows.rename(columns={'index': 'Order', 'story_id': 'FictionId'}, inplace=True)
already_loaded_rows.loc[:, 'FictionId'] = already_loaded_rows['FictionId'].astype(int)
already_loaded_rows.loc[:, 'Order'] = already_loaded_rows['Order'].astype(int)

In [7]:
merged = original_rows.merge(already_loaded_rows, on=['FictionId', 'Order'], how='left', indicator=True)

# Filter for rows that are only in original_rows
original_rows = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

In [8]:
original_rows

,FictionId,content,Order
3,24,"<table style=""border-collapse: collapse; width...",4
354,1157,<p>================================<br> Chapte...,140
1172,1520,"<p><span style=""font-weight: bold"">Character S...",6
2178,2666,"<p style=""text-align: justify""><span style=""te...",41
2201,2666,"<p style=""text-align: justify""><strong><u>Chap...",65
...,...,...,...
1239475,9587,"<p style=""text-align: left"">&nbsp; &nbsp; &nbs...",1
1239854,9624,"<p style=""margin-bottom: 0in; line-height: 150...",3
1242282,99,"<div style=""text-align: center;""><span style=""...",61
1242283,99,"<div style=""text-align: center;""><span style=""...",62


In [9]:
original_rows.to_pickle('./cleaned_normalized_df_no_grouping_v2.pkl')

In [13]:
original_chunks = []

num_rows = len(original_rows)
chunk_size = math.ceil(num_rows / 6)

# break original_rows into 6 pieces
for i in range(1, 7):
    chunk_df = original_rows.iloc[(i-1)*chunk_size:i*chunk_size]
    original_chunks.append(chunk_df)

In [14]:
# save each chunk to a pickle in ./deduped_chunks
for i in range(1, 7):
    original_chunks[i-1].to_pickle(f'./deduped_chunks/original_rows_{i}.pkl')